# Introduction to Variables

A TensorFlow **variable** is the recommended way to represent shared, persistent state your program manipulates. This guide covers how to create, update, and manage instances of `tf.Variable` in TensorFlow.

Variables are created and tracked via the `tf.Variable` class. A `tf.Variable` represents a tensor whose value can be changed by running ops on it. Specific ops allow you to read and modify the values of this tensor. Higher level libraries like `tf.keras` use `tf.Variable` to store model parameters. This guide covers how to create, update, and manage instances of `tf.Variable` in TensorFlow.

## Setup

In [1]:
import tensorflow as tf
# tf.debugging.set_log_device_placement(True) # To see which device is used for each operation

2023-03-02 12:56:46.271766: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX512_VNNI
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Create a variable

A variable is created by passing an initial value to the `tf.Variable` constructor. The initial value defines the type and shape of the variable. After construction, the type and shape of the variable are fixed. The value can be read using the `read_value` method.

In [5]:
my_tensor = tf.constant([[1.0, 2.0], [3.0, 4.0]])
my_variable = tf.Variable(my_tensor)
bool_variable = tf.Variable([False, False, False, True])
complex_variable = tf.Variable([5 + 4j, 6 + 1j])

print("Tensor: ", my_tensor)
print("\nVariable: ", my_variable)
print("\nBool variable: ", bool_variable)
print("\nComplex variable: ", complex_variable)

Tensor:  tf.Tensor(
[[1. 2.]
 [3. 4.]], shape=(2, 2), dtype=float32)

Variable:  <tf.Variable 'Variable:0' shape=(2, 2) dtype=float32, numpy=
array([[1., 2.],
       [3., 4.]], dtype=float32)>

Bool variable:  <tf.Variable 'Variable:0' shape=(4,) dtype=bool, numpy=array([False, False, False,  True])>

Complex variable:  <tf.Variable 'Variable:0' shape=(2,) dtype=complex128, numpy=array([5.+4.j, 6.+1.j])>


A variable looks and acts like a tensor, and, in fact, is a data structure backed by a `tf.Tensor`. Like tensors, they have a dtype and a shape, and can be exported to NumPy.

In [6]:
print("Shape: ", my_variable.shape)
print("DType: ", my_variable.dtype)
print("As NumPy: ", my_variable.numpy())

Shape:  (2, 2)
DType:  <dtype: 'float32'>
As NumPy:  [[1. 2.]
 [3. 4.]]


Most tensor operations work on variables as expected, although variables **cannot be reshaped**.

In [7]:
print("A variable:", my_variable)
print("\nViewed as a tensor:", tf.convert_to_tensor(my_variable))
print("\nIndex of highest value:", tf.math.argmax(my_variable))

# This creates a new tensor; it does not reshape the variable.
print("\nCopying and reshaping: ", tf.reshape(my_variable, [1,4]))

A variable: <tf.Variable 'Variable:0' shape=(2, 2) dtype=float32, numpy=
array([[1., 2.],
       [3., 4.]], dtype=float32)>

Viewed as a tensor: tf.Tensor(
[[1. 2.]
 [3. 4.]], shape=(2, 2), dtype=float32)

Index of highest value: tf.Tensor([1 1], shape=(2,), dtype=int64)

Copying and reshaping:  tf.Tensor([[1. 2. 3. 4.]], shape=(1, 4), dtype=float32)


As noted above, variables are backed by tensors. You can reassign the variable using `tf.Variable.assign`. Calling **assign** does not (usually) allocate a new tensor; instead, the existing tensor's memory is reused.

In [25]:
a = tf.Variable([[1, 2], [3, 4]])

# This will keep the same underlying tensor and the same dtype.
a.assign([[0, 0], [0, 0]])
print(a)

# Not allowed as it resizes the variable: 
try:
  a.assign([1, 2, 3, 4])
except Exception as e:
  print(f"{type(e).__name__}: {e}")

# This is not allowed as it would change the dtype.
try:
    a.assign([[1.1, 2.2], [3.3, 4.4]])
except Exception as e:
    print(f"{type(e).__name__}: {e}")

# However, if it is possible to cast the new value to the same dtype, it will work.
b = tf.Variable([[1.1, 2.2], [3.3, 4.4]])
b.assign([[1, 2], [3, 4]])
print(b)

# You can also assign_add() and assign_sub() to a variable.
print(a.assign_add([[1, 1], [1, 1]]).numpy())


<tf.Variable 'Variable:0' shape=(2, 2) dtype=int32, numpy=
array([[0, 0],
       [0, 0]], dtype=int32)>
ValueError: Cannot assign value to variable ' Variable:0': Shape mismatch.The variable shape (2, 2), and the assigned value shape (4,) are incompatible.
TypeError: Cannot convert [[1.1, 2.2], [3.3, 4.4]] to EagerTensor of dtype int32
<tf.Variable 'Variable:0' shape=(2, 2) dtype=float32, numpy=
array([[1., 2.],
       [3., 4.]], dtype=float32)>
[[1 1]
 [1 1]]


If you use a variable like a tensor in operations, you will usually operate on the backing tensor.

Creating new variables from existing variables **duplicates** the backing tensors. Two variables will not share the same memory.

In [26]:
a = tf.Variable(1.0)
b = tf.Variable(a)
a.assign(2.0)

print(a.numpy(), b.numpy()) # They are different!

2.0 1.0


## Lifecycle of a variable, naming and watching

In Python-based TensorFlow, `tf.Variable` instance have the same **lifecycle** as other Python objects. When there are no references to a variable it is automatically deallocated.

Variables can also be **named** which can help you track and debug them. You can give two variables the same name.

In [34]:
a = tf.Variable(my_tensor, name="my_variable")
b = tf.Variable(my_tensor+1, name="my_variable")

# They have same name but different values
print(a.name, b.name)
print(a==b)

my_variable:0 my_variable:0
tf.Tensor(
[[False False]
 [False False]], shape=(2, 2), dtype=bool)


Variable names are preserved when saving and loading models. By default, variables in models will acquire unique variable names automatically, so you don't need to assign them yourself unless you want to.

Although variables are important for differentiation, some variables will not need to be differentiated. You can **turn off gradients** for a variable by setting `trainable` to `false` at creation. An example of a variable that would not need gradients is a training step counter.

In [36]:
step_counter = tf.Variable(1, trainable=False)
print(step_counter)

<tf.Variable 'Variable:0' shape=() dtype=int32, numpy=1>


## Placing variables and tensors

For better performance, TensorFlow will attempt to place tensors and variables on the fastest device compatible with its dtype. This means most variables are placed on a GPU if one is available.

However, you can override this. In this snippet, place a float tensor and a variable on the CPU, even if a GPU is available. By turning on device placement logging (see **Setup**), you can see where the variable is placed.

Although manual placement works, using [distribution strategies](https://www.tensorflow.org/guide/distributed_training) can be a more convenient and scalable way to optimize your computation.

If you run this notebook on different backends with and without a GPU you will see different logging. Note that logging device placement must be turned on at the start of the session.

In [37]:
tf.debugging.set_log_device_placement(True)

In [39]:
with tf.device("CPU:0"):
    a = tf.Variable([1.0, 2.0, 3.0])
    print(a.device)

with tf.device("GPU:0"):
    b = tf.Variable([1.0, 2.0, 3.0])
    print(b.device)

/job:localhost/replica:0/task:0/device:CPU:0
/job:localhost/replica:0/task:0/device:CPU:0


It's possible to set the location of a variable or tensor on one device and do the computation on another device. This will introduce delay, as data needs to be copied between the devices.

You might do this, however, if you had multiple GPU workers but only want one copy of the variables

In [40]:
with tf.device('CPU:0'):
  a = tf.Variable([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
  b = tf.Variable([[1.0, 2.0, 3.0]])

with tf.device('GPU:0'):
  # Element-wise multiply
  k = a * b

print(k)

tf.Tensor(
[[ 1.  4.  9.]
 [ 4. 10. 18.]], shape=(2, 3), dtype=float32)


To see the full list of devices available to TensorFlow, run the following:

In [45]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]